In [ ]:
#Dette er bare rett fra KI
#Ville bare se om programe kjørte 

import os
import requests
import json
import pandas as pd
from pandasql import sqldf

def hent_data_fra_api(url, headers=None):
    """Henter JSON-data fra et API og returnerer det som en Python-dictionary."""
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Feil ved henting av data: {response.status_code}")
        return None

def lagre_json_fil(data, filsti):
    """Lagrer JSON-data til en fil."""
    with open(filsti, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def json_til_dataframe(data):
    """Konverterer JSON-data til en DataFrame, og flater ut nested dictionaries."""
    df = pd.json_normalize(data['properties']['timeseries'])  # Flater ut JSON

    # Erstatt '.' med '_' i kolonnenavnene for SQLite-kompatibilitet
    df.columns = [col.replace(".", "_") for col in df.columns]

    return df

def analyser_data_med_sql(df, query):
    """Bruker SQL på Pandas DataFrame for å analysere data."""
    return sqldf(query, locals())

def main():
    # Definer API-informasjon
    url = "https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=60.10&lon=9.58"
    headers = {"User-Agent": "MiljodataAnalyse/1.0"}
    
    # Opprett katalog for lagring av data
    datakatalog = "data"
    os.makedirs(datakatalog, exist_ok=True)
    
    # Hent data fra API
    data = hent_data_fra_api(url, headers)
    if data:
        json_fil = os.path.join(datakatalog, "weather_data.json")
        csv_fil = os.path.join(datakatalog, "weather_data.csv")
        
        # Lagre JSON-data
        lagre_json_fil(data, json_fil)
        
        # Konverter JSON til Pandas DataFrame og flatten
        df = json_til_dataframe(data)
        df.to_csv(csv_fil, index=False)  # Lagre til CSV
        
        # Analyser med SQL (bruker nytt formatert navn på kolonnen)
        query = "SELECT time, data_instant_details_air_temperature FROM df LIMIT 5"
        df_resultat = analyser_data_med_sql(df, query)
        
        print("Første 5 rader av data med temperatur:")
        print(df_resultat)
    else:
        print("Kunne ikke hente data.")

if __name__ == "__main__":
    main()
      
  



Første 5 rader av data med temperatur:
                   time  data_instant_details_air_temperature
0  2025-03-07T12:00:00Z                                   8.2
1  2025-03-07T13:00:00Z                                   9.1
2  2025-03-07T14:00:00Z                                   9.1
3  2025-03-07T15:00:00Z                                   8.4
4  2025-03-07T16:00:00Z                                   7.2
